In [ ]:
import pandas as pd
names = ['Tweet', 'Label']
df = pd.read_csv('/content/fnn_train.csv', sep=',', names=names)
df.head()

In [ ]:
df_val = pd.read_csv('/content/fnn_test.csv', sep=',', names=names)

In [ ]:
!pip install "tensorflow==1.15.0"

In [ ]:
!pip install tensorflow-hub

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

In [ ]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [ ]:
def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [ ]:
df.shape

In [ ]:
list_train = [df[i:i+10] for i in range(0,df.shape[0],10)]
list_test = [df_val[i:i+10] for i in range(0,df_val.shape[0],10)]

In [ ]:
len(list_train)

In [ ]:
elmo_train = [elmo_vectors(x['Tweet']) for x in list_train]
elmo_test = [elmo_vectors(x['Tweet']) for x in list_test]

In [ ]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)

In [ ]:
from sklearn.model_selection import train_test_split

xtrain, xvalid, ytrain, yvalid = train_test_split(elmo_train_new, 
                                                  df['label'],  
                                                  random_state=42, 
                                                  test_size=0.2)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

lreg = LogisticRegression()
lreg.fit(xtrain, ytrain)

In [ ]:
preds_valid = lreg.predict(xvalid)

In [ ]:
preds_test = lreg.predict(elmo_test_new)